# Prepare Data
1. load power df
1. reindex with subject and condition
2. load behavioral df
3. melt behavioral on condition
4. join power and behavioral 
5. drop unnecessary rows and columns

In [14]:
# setup 
import pandas as pd
 
# open behavioral data and ids map
bh = pd.read_csv('data/behavioral_data/archived/plb_hyp_data.csv', index_col='index')
ids_map = pd.read_excel('docs/ids_map.xlsx', header=1, index_col='behavioral_id')
ids_map = ids_map.drop_duplicates('bids_id')
ids_map = ids_map[['bids_id']]
ids_map['bids_id'] = ids_map['bids_id'].apply(lambda x:str(x).zfill(2))
bh = bh.join(ids_map, how='right')
bh = bh.melt(
    id_vars=['procedure_type_1', 'procedure_type_2', 'procedure_type_3', 'procedure_type_4', 'bids_id',
             'description_type_1', 'description_type_2', 'description_type_3', 'description_type_4',
             'experiences_freetext_1', 'experiences_freetext_2', 'experiences_freetext_3', 'experiences_freetext_4'],
    value_vars=['hypnosis_depth_1', 'hypnosis_depth_2', 'hypnosis_depth_3', 'hypnosis_depth_4'])
bh['session'] = bh['variable'].apply(lambda x:x.split('_')[2])
bh['procedure'] = bh.apply(lambda r: r['procedure_type_'+r['session']], axis=1)
bh['description'] = bh.apply(lambda r: r['description_type_'+r['session']], axis=1)
bh['freetext'] = bh.apply(lambda r: r['experiences_freetext_'+r['session']], axis=1)

bh = bh[['bids_id', 'value', 'procedure', 'description', 'freetext', 'session']].sort_values(by=['bids_id', 'session']).set_index('bids_id')
bh = bh.rename(columns={'value':'hypnosis_depth'})
bh.reset_index(inplace=True)

# bh.to_csv('data/behavioral_data/behavioral_data_with_freetext.csv', index=0)

In [4]:
# open power data
power = pd.read_excel('docs/psds_new.xlsx', index_col='Unnamed: 0', header=1)
power.columns = power.columns.str.replace('lower_gamma', 'lowergamma')
power[['bids_id', 'condition']] = power.index.to_series().apply(lambda x:x.split('-')).apply(pd.Series)
power['session'] = power['condition'].apply(lambda x:x[-1])
power.reset_index(drop=True, inplace=True)
power = pd.merge(bh, power, how='right', on=['session', 'bids_id'], right_index=False)
power = power.sort_values(by=['bids_id', 'session', 'condition']).reset_index(drop=True)
power.insert(1, 'condition', power.pop('condition'))
# power.to_csv('data/classification_dataset.csv')
power.head()

In [ ]:
# debug
# power = power.query('condition.str.contains("experience")', engine='python')

In [6]:
DATA = pd.read_csv(f'data/classification_datasets/tobedeleted/plv_source.csv')
DATA.sort_values(by=['bids_id', 'session'], inplace=True)
DATA.drop(columns=['bids_id', 'session', 'condition','Unnamed: 0.1',
                   'Unnamed: 0'], inplace=True)
# DATA.to_csv(f'data/classification_datasets/plv_source.csv', index=False)

[]